# Werkje gegevensbanken 2018 - DEEL 04

## Inleiding

Het vierde en laatste deel van het werkje bestaat uit twee componenten: 
1. Optimalisatie
2. Visualisatie

Bij dit deel van het werkje laten we jullie veel vrijheid om tot een oplossing te komen. Er bestaat niet zoiets als de unieke correcte oplossing, het is dus de bedoeling dat jullie zelf een strategie uitdokteren en deze achteraf kunnen verdedigen. Leg in deze notebook kort uit welke keuzes je maakt en waarom, dit zal helpen bij de verdediging. Beschouw dit als een soort mini-verslag tussen de code door.

### Indienen + Evaluatiemoment

** EVALULATIEMOMENT**: DE OEFENZITTING IN DE WEEK VAN 14/05/2017-18/05/2017

Het evaluatiemoment van het **volledige** werkje vindt plaats tijdens **de oefenzitting in de week van 14/05/2017-18/05/2017**. Prof. Berendt en dr. Bogaerts komen langs om enkele vragen te stellen over jullie oplossingen. Dit evaluatiemoment omvat ook een demonstratie van jullie resultaat, bijvoorbeeld: aantonen dat jullie optimalisatie van deel 4 van het werkje effectief een verbetering is t.o.v. het originele geval. Teneinde die demonstratie vlot te doen verlopen vragen we jullie om **per groep (minstens) 1 laptop mee te nemen waarop jullie oplossingen (i.e. de ingevulde notebooks) goed functioneren**. Deze laptop beschikt dus ook over XAMPP (inclusief de gegevensbank van dit werkje, uiteraard), gezien de queries in de notebooks ook moeten werken.

**INDIENDEADLINE**: ZONDAG 13 MEI om 23.59

** HOE INDIENEN**:
    1. Upload je ingevulde notebook als werkje_04_groep_XX.ipynb EN werkje_04_groep_XX.html 
    2. Vervang XX door je groepsnummer.
    3. Uploaden doe je op Toledo in je groepsfolder.
    
Zorg ervoor dat de .html file zeker ook de output van de visualisatie bevat, zodat wij deze gemakkelijk kunnen bekijken en niet moeten reproduceren door jullie .ipynb te runnen! Op die manier kunnen we zeker zijn dat we evalueren wat jullie geproduceerd hebben.

### Nuttige packages en functies inladen

Enkele nuttige packages en functies inladen die verderop van pas zullen komen.

In [ ]:
# Benodigde packages
import json            # Package om .json files in te laden (bvb kolomnamen zijn zo opgeslagen)
import getpass         # Package om een paswoordveldje te genereren.
import mysql.connector # MySQL package
import numpy as np
import pandas as pd    # Populaire package voor data-verwerking
import sys
import os
import time

In [ ]:
sys.version_info       # Check python versie, wij veronderstellen 3.6

### Interageren met een gegevensbank

In [ ]:
def verbind_met_GB(username, hostname, gegevensbanknaam):
    """
    Maak verbinding met een externe gegevensbank
    
    :param  username:          username van de gebruiker, string
    :param  hostname:          naam van de host, string.
                               Dit is in het geval van een lokale server gewoon 'localhost'
    :param  gegevensbanknaam:  naam van de gegevensbank, string.
    :return connection:        connection object, dit is wat teruggeven wordt 
                               door connect() methods van packages die voldoen aan de DB-API
    """
    
    password = getpass.getpass() # Genereer vakje voor wachtwoord in te geven
    
    connection = mysql.connector.connect(host=hostname,
                                         user=username,
                                         passwd=password,
                                         db=gegevensbanknaam)
    return connection


def check_perfect_match(df1, df2):
    """
    Functie om te checken of 2 DataFrames gelijk zijn.
    """
    check = df1.equals(df2)
    return check


def run_query(connection, query):
    """
    Voer een query uit op een reeds gemaakte connectie, geeft het resultaat van de query terug
    """
    
    # Making a cursor and executing the query
    cursor = connection.cursor()
    cursor.execute(query)
    
    # Collecting the result and casting it in a pd.DataFrame
    res = cursor.fetchall()
    
    return res


def res_to_df(query_result, column_names):
    """
    Giet het resultaat van een uitgevoerde query in een 'pandas dataframe'
    met vooraf gespecifieerde kolomnamen.
    
    Let op: Het resultaat van de query moet dus exact evenveel kolommen bevatten
    als kolomnamen die je meegeeft. Als dit niet het geval is, is dit een indicatie
    dat je oplossing fout is. (Gezien wij de kolomnamen van de oplossing al cadeau doen)
    
    """
    df = pd.DataFrame(query_result, columns=column_names)
    return df

## Optimalisatie

### Opgave

Het doel van het eerste deel van deze taak is het optimaliseren van onderstaande query.

Een goede oplossing van deze eerste opgave voldoet aan volgende criteria:
* Verzin een optimalisatie van deze query
* De runtime van de optimalisatie is significant minder dan die van de originele oplossing (een paar procent is te weinig)
* Je bent in staat om uit te leggen WAT je doet en WAAROM je dat gedaan hebt en WAAROM het werkt..
    * Het EXPLAIN statement in MySQL is een goede gids om je hierbij te helpen. 
    * Je schrijft in deze notebook al kort de redeneringen op die achter jullie optimalisatie zitten (dit zal nuttig zijn bij de mondelinge verdediging).

Hieronder vinden jullie de query die te optimaliseren is. Wijzig deze dus niet.

**Beschrijving**

Het resultaat van deze functie is een Pandas DataFrame dat voor een gegeven *jaar_1* een aantal statistieken van alle staten bevat waarbij de gemiddelde lengte van alle spelers geboren in die staat en opgenomen in de hall of fame na *jaar_2* groter is dan *lengte*.

Voor die staten moet de tabel de volgende statistieken bevatten: het gemiddelde gewicht, de gemiddelde lengte, het gemiddeld aantal batting homeruns, en het gemiddeld aantal pitching saves van alle spelers (geboren in die staat) die in de hall of fame zijn opgenomen na *jaar_2*.

Sorteer oplopend alfabetisch op staat.

Nb. Lengte is uitgedrukt in inches.

In [ ]:
column_names = ['state', 'avg_weight', 'avg_height', 'avg_homeruns', 'avg_saves']

In [ ]:
# Dit is de originele query die geoptimaliseerd dient te worden. Niet wijzigen!
def query_to_optimize(connection, column_names, jaar_1=2000, jaar_2=1990, lengte=75):
    # Actual query
    query="""
    SELECT m.birthState, AVG(m.weight), AVG(m.height), AVG(bat.HR), AVG(pit.SV)
    FROM Master AS m,
        Pitching AS pit,
        Batting AS bat,
        HallOfFame AS hof
    WHERE pit.yearID = {}
        AND bat.yearID = {}
        AND pit.playerID = m.playerID
        AND bat.playerID = m.playerID
        AND m.playerID = hof.playerID
        AND hof.yearID > {}
    GROUP BY m.birthState
    HAVING AVG(m.height) > {}
    ORDER BY m.birthState ASC;
    """.format(jaar_1, jaar_1, jaar_2, lengte)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

Eerst maken we verbinding met de gegevensbank, zoals jullie al deden in het vorige deel van het werkje.

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In onderstaande cel bekomen we de runtime van de originele query door deze vijf keer te runnen, om tot een betere schatting te komen. Deze conventie van runtimes meten dient behouden te worden.

In [ ]:
timings=[]

# We voeren deze query 5 keer uit om een betere schatting te krijgen van de effectieve runtime.
for i in range(5):
    t1 = time.time() # Start time
    df_orig = query_to_optimize(c, column_names, jaar_1=2000, jaar_2=1990, lengte=75)
    t2 = time.time() # Stop time
    timings.append(t2-t1) 
    
orig_runtime=np.mean(timings) # De uiteindelijke runtime is een gemiddelde van 5 runs.

print('De originele query duurt: ', "{0:.2f}".format(orig_runtime),' seconden')

### Optimalisatie

In deze sectie is het de bedoeling dat je je optimalisatie realiseert. Wat hier al ingevuld staat qua code is enkel om je op weg te helpen.

Je mag zoveel codecellen toevoegen als je nodig acht om je verhaal te vertellen. Leg uit wat je doet, en waarom je dat doet. Denk hierbij zeker aan het EXPLAIN statement zoals gezien in de les. Gebruik EXPLAIN om te identificeren hoe MySQL de query omzet en kijk zo welke effecten je verbeteringen hebben. Leg de resultaten van EXPLAIN kort uit.

Beschouw de uitleg rond je oplossing als een soort mini-verslag dat je code en query uitlegt.

In [ ]:
# Maak hier je eerste optimalisatie
def optim_1(connection, column_names, jaar_1=2000, jaar_2=1990, lengte=75):
    query="""
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

### Controle

De onderstaande codecel vergelijkt de runtime van de optimalisatie met die van de originele query, door een gemiddelde te nemen over 5 runs. Ook wordt er nagegaan dat de resultaten zeker volledig gelijk zijn. Wijzig deze controlewijze niet.

In [ ]:
# Test hier je eerste optimalisatie

timings_optim_1 = []
for i in range(5):
    t1 = time.time() # Start time
    df_optim_1 = optim_1(c, column_names, jaar_1=2000, jaar_2=1990, lengte=75)
    t2 = time.time() # Stop time
    timings_optim_1.append(t2-t1) 
    
optim_1_runtime = np.mean(timings_optim_1) # Runtime optimalisatie 1

# Vergelijken met originele query
diff = orig_runtime-optim_1_runtime # Winst t.o.v. origineel
rel_diff = diff/orig_runtime # Relatieve winst
check = check_perfect_match(df_orig,df_optim_1) # Nagaan of de resultaten exact gelijk zijn.

# Belangrijkste resultaten printen.
print('De optimalisatie duurt: ', "{0:.2f}".format(optim_1_runtime),' seconden')
print('De originele versie duurde: ', "{0:.2f}".format(orig_runtime),' seconden')
print('De netto tijdwinst is dus: ',"{0:.2f}".format(diff),' seconden, oftewel', "{0:.2f}".format(100*rel_diff), '%' )
print('Is het resultaat equivalent? ', check)

## Visualisatie

### Opgave

In het laatste onderdeel van het werkje laten we jullie volledig vrij. Jullie hebben ondertussen geleerd om data uit een gegevensbank op te halen, de volgende stap is om deze data effectief voor iets nuttigs te gebruiken.

Een snelle manier om gegevens overzichtelijk weer te geven is visualisatie. Een goede visualisatie kan zeker in een eerste fase van data-analyse echt al inzichten verschaffen. Wij vragen dus aan jullie een of meerdere visualisaties te realiseren, met data uit de gegevensbank die jullie voor dit werkje gebruikt hebben.

Een goede oplossing van deze opgave voldoet aan volgende criteria:
* Schrijf een query die informatie uit de gegevensbank ophaalt, en in een DataFrame stopt.
* Maak een interessante visualisatie van (een deel van) de data in die DataFrame
* Je beschrijft kort (mini-verslag hier in de notebook) wat je doet, en wat de inzichten zijn die de visualisatie verschaft.
* Je bent in staat om dit mini-verslag ook mondeling te verdedigen, i.e. antwoorden op: wat doe je, waarom doe je dit, en wat was het resultaat.
* 1 visualisatie volstaat, meer mag altijd.

### Nuttige packages

Om jullie al een beetje op weg te helpen, geven we jullie alvast de naam van twee handige en veelvoorkomende visualisatietools die goed werken met pandas DataFrames:

1. seaborn
2. matplotlib

Voor een korte inleiding ivm seaborn kan je hier kijken: https://chrisalbon.com/python/pandas_with_seaborn.html
Voor een korte inleiding ivm matplotlib kan je hier kijken: http://pandas.pydata.org/pandas-docs/stable/visualization.html

Natuurlijk moet je vooraleer je aan zo'n visualisatie kan beginnen, al een pandas DataFrame hebben dat de juiste informatie bevat. De eerste stap zal er dus in bestaan om een goede query te schrijven die de nodige informatie uit de gegevensbank weet te halen, gevolgd door een visualisatie van de DataFrame die uit die query volgt.

In [ ]:
#VISUALISATIE 1
column_names2 = ['weight', 'avg_salary']

def vis_query(connection, column_names):
    query="""
    SELECT m.Weight, AVG(s.Salary)
    FROM salaries as s, Master as m
    WHERE m.PlayerID = s.PlayerID
    GROUP BY(m.PlayerID)
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

vis_pandas = vis_query(c, column_names2)



In [ ]:
%matplotlib inline
vis_pandas['avg_salary'] = vis_pandas['avg_salary'].astype('float')
vis_pandas.dtypes
vis_pandas.plot.scatter(x='avg_salary', y='weight')

In [ ]:
import matplotlib.pyplot as plt
vis_np = np.array(vis_pandas)
#numpy array verliest kolomnaam, 1ste is avg_salary, 2de is weight
plt.scatter(vis_np[:, 0], vis_np[:, 1])

In [ ]:
from sklearn import cluster as cl

for i in range(2, 10, 2):
    plt.figure(tight_layout=True)
    
    plt.subplot(141)
    plt.yscale('log')
    clf = cl.AgglomerativeClustering(i)
    y = clf.fit_predict(vis_np)
    plt.title("Ward-clustering: %d" % i)
    plt.scatter(vis_np[:, 0], vis_np[:, 1], c=y, cmap='viridis')

    plt.subplot(142)
    plt.yscale('log')
    plt.title("KMeans-clustering: %d" % i)
    clf = cl.KMeans(i)
    y = clf.fit_predict(vis_np)
    plt.scatter(vis_np[:, 0], vis_np[:, 1], c=y, cmap='viridis')
    
    plt.subplot(143)
    plt.yscale('log')
    clf = cl.AgglomerativeClustering(i+1)
    y = clf.fit_predict(vis_np)
    plt.title("Ward-clustering: %d" % (i+1))
    plt.scatter(vis_np[:, 0], vis_np[:, 1], c=y, cmap='viridis')

    plt.subplot(144)
    plt.yscale('log')
    plt.title("KMeans-clustering: %d" % (i+1))
    clf = cl.KMeans(i+1)
    y = clf.fit_predict(vis_np)
    plt.scatter(vis_np[:, 0], vis_np[:, 1], c=y, cmap='viridis')
    plt.show()

In [ ]:
plt.figure(tight_layout=True)
plt.yscale('log')
bandwidth = cl.estimate_bandwidth(vis_np)
clf = cl.MeanShift(bin_seeding=True)
y = clf.fit_predict(vis_np)
plt.title("MeanShift-clustering")
plt.scatter(vis_np[:, 0], vis_np[:, 1], c=y, cmap='viridis')

### 2de visualisatie:
#### avg homeruns vs avg salaris
Merk op dat tupels met bijna 0 avg homeruns niet meegenomen worden en merk op dat avg homeruns * 10 gedaan wordt, arbitrair

In [ ]:
#VISUALISATIE 2
column_names3 = ['avg_homeruns', 'avg_salary']

def vis_query(connection, column_names):
    query="""
    SELECT *
    FROM (
    SELECT AVG(b.HR)*10 as ah, AVG(s.salary)
    FROM salaries as s, Master as m, battingpost as b
    WHERE m.PlayerID = s.PlayerID AND m.playerID = b.playerID
    GROUP BY(m.PlayerID)
) as sub
    WHERE sub.ah>0.01
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

vis2_pd = vis_query(c, column_names3)
vis2_pd.head()

In [ ]:
vis2_pd['avg_salary'] = vis2_pd['avg_salary'].astype('float')
vis2_pd['avg_homeruns'] = vis2_pd['avg_homeruns'].astype('float')
vis2_pd.dtypes

In [ ]:
#from mpl_toolkits.mplot3d import Axes3D
vis2_np = np.array(vis2_pd)
#numpy array verliest kolomnaam, 1ste is weight, 2de avg_salary, 3de avg_homeruns
plt.figure()
plt.yscale('log')
#plt.subplot(111, projection ='3d')
plt.scatter(vis2_np[:, 0], vis2_np[:, 1])

In [ ]:
plt.figure(tight_layout=True)
plt.yscale('log')
bandwidth = cl.estimate_bandwidth(vis2_np, n_samples = 1000)
clf = cl.MeanShift(bin_seeding=True)
y = clf.fit_predict(vis2_np)
plt.title("MeanShift-clustering")
plt.scatter(vis2_np[:, 0], vis2_np[:, 1], c=y, cmap='viridis')